In [2]:
!pip install google-generativeai python-dotenv pandas


In [5]:

import os
import google.generativeai as genai
from dotenv import load_dotenv

# Load API key
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Upload CSV file
file = genai.upload_file("ArtificialData\\users.csv")

print("Uploaded file:", file.display_name)
print("File URI:", file.uri)  # Use this to reference in prompts


Uploaded file: users.csv
File URI: https://generativelanguage.googleapis.com/v1beta/files/bnlybjiywznf


In [7]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

# Load API key
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Upload CSV correctly (force mime_type as text/csv)
file = genai.upload_file(
    path="ArtificialData/users.csv",
    mime_type="text/csv"
)

print("Uploaded file:", file.display_name)
print("File URI:", file.uri)

# Query Gemini with the uploaded file
model = genai.GenerativeModel("gemini-1.5-flash")

response = model.generate_content([
    "Answer questions based on this dataset of users:",
    file
])

print(response.text)


Uploaded file: users.csv
File URI: https://generativelanguage.googleapis.com/v1beta/files/us7e1ggb454x
Here are the answers based on the provided dataset:

**1. How many users are in the dataset?**

There are 11 users in the dataset.

**2. What is the most common role?**

The most common role is "Operator", with 7 users.

**3. When was the most recent login?**

The most recent login was on 2025-09-12T20:00:00Z.

**4. When was the earliest login?**

The earliest login was on 2025-09-12T11:00:00Z.

**5.  What is the userId of the user with the role "Admin"?**

The userId of the user with the role "Admin" is user002.

**6. How many Maintenance Managers are there?**

There are 2 Maintenance Managers.

**7. What is the average time (in days) between account creation (`createdAt`) and last login (`lastLogin`)?**

To calculate this accurately, we need to convert the date and time strings to a numerical representation (e.g., Unix timestamps or datetime objects in a programming language like Py

In [8]:
model = genai.GenerativeModel("gemini-1.5-flash")

response = model.generate_content([
    "User ID of supervisor and admin users in the dataset:",
    file
])

print(response.text)

The supervisor user ID is `user003` and the admin user ID is `user002`.



In [9]:
model = genai.GenerativeModel("gemini-1.5-flash")

response = model.generate_content([
    "Trains which are out of service and trains which have a job card assigned to them in the dataset:",
    file
])

print(response.text)

The provided dataset only contains user information (userId, lastLogin, createdAt, role).  There is no information about trains, their service status, or assigned job cards.  Therefore, it's impossible to answer your question using this data.  You need a dataset that includes train-related information.



In [ ]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

# Load API key
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Get all CSV files in the folder
csv_folder = "ArtificialData"
csv_files = [os.path.join(csv_folder, f) for f in os.listdir(csv_folder) if f.endswith(".csv")]

# Upload all CSV files
files = [genai.upload_file(path=f, mime_type="text/csv") for f in csv_files]

model = genai.GenerativeModel("gemini-1.5-flash")

response = model.generate_content([
    "Which Train is due cleaning and which Train is out of service in the dataset:?",
    *files  # unpack list of files
])

print(response.text)

Error generating content: 403 You do not have permission to access the File ogvi0dttsiiq or it may not exist.
Note: Could not clean up uploaded files


In [ ]:
# Gemini AI Prompt Template for Metro CSV-based Q&A

prompt = """
You are an AI assistant that answers metro-related questions based on CSV files from the ArtificialData folder.

Follow this JSON schema strictly in your output:

{
    "csv_file_path": "Path to the CSV file in ArtificialData that contains the data to answer the question.",
    "question": "The user-defined metro-related question to answer based on the CSV data.",
    "answer": "Plain English explanation or answer generated from the CSV data.",
    "question_type": "Type of question based on the CSV content. One of ['summary', 'filtering', 'aggregation', 'transformation', 'comparison'].",
    "filters": {
        "column_name": "Optional: Name of the column to filter.",
        "value": "Optional: Value to filter the column by."
    },
    "pandas_code": "Suggested pandas code to fetch the answer from the CSV file."
}

Instructions:
1. Read the CSV file specified in 'csv_file_path' from the ArtificialData folder.
2. Understand the user's 'question'.
3. Provide a clear 'answer' in plain English.
4. Specify the 'question_type'.
5. If needed, suggest filters in 'filters'.
6. Provide working pandas code in 'pandas_code' to fetch the answer from the CSV.

Example output:

{
    "csv_file_path": "ArtificialData/trainsets.csv",
    "question": "Which trains are currently out of service?",
    "answer": "Trainsets TS-01 and TS-03 are currently out of service.",
    "question_type": "filtering",
    "filters": {
        "column_name": "service_status",
        "value": "out of service"
    },
    "pandas_code": "import pandas as pd\ndf = pd.read_csv('ArtificialData/trainsets.csv')\nout_of_service = df[df['service_status'] == 'out of service']['trainset_id'].tolist()\nout_of_service"
}

Now, generate the JSON output for the user's question.
"""

In [21]:
import json
import google.generativeai as genai

# Configure the API key (you'll need to set this)
# genai.configure(api_key="YOUR_API_KEY_HERE")

# Define JSON schema
schema = {
    "type": "object",
    "properties": {
        "csv_file_path": {"type": "string"},
        "question": {"type": "string"},
        "answer": {"type": "string"},
        "question_type": {"type": "string", "enum": ["summary", "filtering", "aggregation", "transformation", "comparison"]},
        "filters": {
            "type": "object",
            "properties": {
                "column_name": {"type": "string"},
                "value": {"type": "string"}
            }
        },
        "pandas_code": {"type": "string"}
    },
    "required": ["csv_file_path", "question", "answer", "question_type"]
}

# Initialize the model
model = genai.GenerativeModel("gemini-1.5-flash")

# Create a more detailed prompt for better results
prompt = """
Please analyze the following question about a train dataset and provide a structured JSON response:

Question: "Which Train is due cleaning and which Train is out of service in the dataset?"

Please provide the response in the following JSON format:
{
    "csv_file_path": "path/to/your/dataset.csv",
    "question": "Which Train is due cleaning and which Train is out of service in the dataset?",
    "answer": "Your analysis of trains due for cleaning and out of service",
    "question_type": "filtering",
    "filters": {
        "column_name": "status or maintenance_status",
        "value": "due_cleaning or out_of_service"
    },
    "pandas_code": "Python pandas code to filter the data"
}
"""

try:
    # Generate structured JSON content
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            response_mime_type="application/json",
            response_schema=schema
        )
    )
    
    # Safely parse JSON output
    try:
        structured_output = json.loads(response.text)
        print("Successfully generated structured output:")
        print(json.dumps(structured_output, indent=2))
    except json.JSONDecodeError as e:
        print(f"Error: The model did not return valid JSON. Error: {e}")
        print("Raw response:", response.text)
        structured_output = None
        
except Exception as e:
    print(f"Error generating content: {e}")
    structured_output = None

Successfully generated structured output:
{
  "csv_file_path": "train_dataset.csv",
  "question": "Which Train is due cleaning and which Train is out of service in the dataset?",
  "answer": "To answer this question, we need to analyze the 'status' or 'maintenance_status' column of the train dataset.  Trains with a status of 'due_cleaning' are due for cleaning, and trains with a status of 'out_of_service' are out of service.  The specific train IDs will depend on the data in the dataset.",
  "question_type": "filtering",
  "filters": {
    "column_name": "status",
    "value": "due_cleaning,out_of_service"
  },
  "pandas_code": "import pandas as pd\ndata = pd.read_csv('train_dataset.csv')\ndue_for_cleaning = data[data['status'].isin(['due_cleaning'])]\nout_of_service = data[data['status'].isin(['out_of_service'])]\nprint('Trains due for cleaning:\\n', due_for_cleaning)\nprint('\\nTrains out of service:\\n', out_of_service)"
}


In [26]:
import os
import json
import google.generativeai as genai
from dotenv import load_dotenv

# Load API key
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Define JSON schema
schema = {
    "type": "object",
    "properties": {
        "csv_files_analyzed": {
            "type": "array",
            "items": {"type": "string"}
        },
        "question": {"type": "string"},
        "answer": {"type": "string"},
        "question_type": {"type": "string", "enum": ["summary", "filtering", "aggregation", "transformation", "comparison"]},
        "trains_due_cleaning": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "train_id": {"type": "string"},
                    "train_name": {"type": "string"},
                    "status": {"type": "string"},
                    "last_cleaned": {"type": "string"},
                    "source_file": {"type": "string"}
                }
            }
        },
        "trains_out_of_service": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "train_id": {"type": "string"},
                    "train_name": {"type": "string"},
                    "status": {"type": "string"},
                    "reason": {"type": "string"},
                    "source_file": {"type": "string"}
                }
            }
        },
        "summary": {
            "type": "object",
            "properties": {
                "total_trains_due_cleaning": {"type": "integer"},
                "total_trains_out_of_service": {"type": "integer"},
                "total_files_analyzed": {"type": "integer"}
            }
        },
        "pandas_code": {"type": "string"}
    },
    "required": ["csv_files_analyzed", "question", "answer", "question_type", "summary"]
}

def upload_csv_files_once(csv_folder="ArtificialData", file_refs_path="uploaded_file_refs.json"):
    """Upload CSV files once and save their references"""
    
    # Check if we already have uploaded file references
    if os.path.exists(file_refs_path):
        print("Loading existing file references...")
        with open(file_refs_path, 'r') as f:
            file_data = json.load(f)
        
        # Verify files still exist on Google's servers
        try:
            files = []
            for file_info in file_data:
                file_obj = genai.get_file(file_info['name'])
                files.append(file_obj)
            print(f"Successfully loaded {len(files)} existing uploaded files")
            return files
        except Exception as e:
            print(f"Some files may have expired or been deleted: {e}")
            print("Re-uploading files...")
    
    # Upload files for the first time or re-upload if needed
    csv_files = [os.path.join(csv_folder, f) for f in os.listdir(csv_folder) if f.endswith(".csv")]
    
    if not csv_files:
        print(f"No CSV files found in {csv_folder}")
        return []
    
    print(f"Uploading {len(csv_files)} CSV files...")
    files = []
    file_refs = []
    
    for csv_file in csv_files:
        try:
            uploaded_file = genai.upload_file(path=csv_file, mime_type="text/csv")
            files.append(uploaded_file)
            file_refs.append({
                'name': uploaded_file.name,
                'display_name': uploaded_file.display_name,
                'local_path': csv_file
            })
            print(f"Uploaded: {csv_file}")
        except Exception as e:
            print(f"Failed to upload {csv_file}: {e}")
    
    # Save file references for future use
    with open(file_refs_path, 'w') as f:
        json.dump(file_refs, f, indent=2)
    
    print(f"Successfully uploaded and saved references for {len(files)} files")
    return files

def analyze_trains(files, question="Which Train is due cleaning and which Train is out of service in the dataset?"):
    """Analyze the uploaded CSV files with the given question"""
    
    model = genai.GenerativeModel("gemini-1.5-flash")
    
    prompt = f"""
    Analyze the uploaded CSV datasets to answer: "{question}"

    Please examine all the CSV files and provide a structured JSON response with the following information:
    1. List train IDs/names that are due for cleaning (simple array of strings)
    2. List train IDs/names that are out of service (simple array of strings)
    3. Provide detailed information for each train in separate "train_details" section
    4. Provide a summary count

    Structure your response as follows:
    - trains_due_cleaning: Simple array of train identifiers
    - trains_out_of_service: Simple array of train identifiers  
    - train_details: Separate section with detailed information for each train
    - summary: Count totals

    Please respond ONLY with valid JSON that matches the specified schema. Do not include any markdown formatting or additional text.
    """
    
    try:
        response = model.generate_content(
            [prompt, *files],
            generation_config=genai.types.GenerationConfig(
                response_mime_type="application/json",
                response_schema=schema
            )
        )
        
        structured_output = json.loads(response.text)
        return structured_output
        
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON response. Error: {e}")
        print("Raw response:", response.text)
        return None
    except Exception as e:
        print(f"Error generating content: {e}")
        return None

def list_uploaded_files():
    """List all currently uploaded files"""
    try:
        files = list(genai.list_files())
        print(f"\nCurrently uploaded files ({len(files)}):")
        for file in files:
            print(f"- {file.display_name} (ID: {file.name})")
    except Exception as e:
        print(f"Error listing files: {e}")

def cleanup_files(file_refs_path="uploaded_file_refs.json"):
    """Clean up uploaded files and remove references"""
    if os.path.exists(file_refs_path):
        with open(file_refs_path, 'r') as f:
            file_data = json.load(f)
        
        for file_info in file_data:
            try:
                genai.delete_file(file_info['name'])
                print(f"Deleted: {file_info['display_name']}")
            except Exception as e:
                print(f"Could not delete {file_info['display_name']}: {e}")
        
        os.remove(file_refs_path)
        print("Cleaned up file references")
    else:
        print("No file references found to clean up")

# Main execution
if __name__ == "__main__":
    # Upload files once (or load existing references)
    files = upload_csv_files_once()
    
    if not files:
        print("No files available for analysis")
        exit(1)
    
    # Now you can run multiple analyses without re-uploading
    print("\n" + "="*60)
    print("RUNNING ANALYSIS...")
    print("="*60)
    
    # Analysis 1
    result = analyze_trains(files)
    if result:
        print("\nSTRUCTURED JSON OUTPUT:")
        print(json.dumps(result, indent=2))
        
        # Save result
        with open("train_analysis_result.json", "w") as f:
            json.dump(result, f, indent=2)
    
    # You can run more analyses here without re-uploading:
    # result2 = analyze_trains(files, "What is the average age of trains in the dataset?")
    
    print("\n" + "="*60)
    print("Files remain uploaded for future use!")
    print("Run this script again to reuse the same files.")
    print("To clean up files, uncomment the line below:")
    print("="*60)
    
    # Uncomment to clean up files when done:
    # cleanup_files()
    
    # List currently uploaded files
    list_uploaded_files()

Loading existing file references...
Successfully loaded 6 existing uploaded files

RUNNING ANALYSIS...

STRUCTURED JSON OUTPUT:
{
  "csv_files_analyzed": [
    "train_data.csv",
    "cleaning_schedule.csv",
    "train_status.csv"
  ],
  "question": "Which Train is due cleaning and which Train is out of service in the dataset?",
  "answer": "Trains due for cleaning: TS004, TS005, TS007, TS008, TS013, TS014, TS015, TS021, TS022, TS025. Trains out of service: TS010, TS020",
  "question_type": "filtering",
  "summary": {
    "total_files_analyzed": 3,
    "total_trains_due_cleaning": 10,
    "total_trains_out_of_service": 2
  },
  "trains_due_cleaning": [
    {
      "train_id": "TS004",
      "train_name": "Hyundai"
    },
    {
      "train_id": "TS005",
      "train_name": "Paytm"
    },
    {
      "train_id": "TS007",
      "train_name": "Pepsi"
    },
    {
      "train_id": "TS008",
      "train_name": "Hyundai"
    },
    {
      "train_id": "TS013",
      "train_name": "ICICI"
   